In [3]:
from collections import defaultdict
from functools import lru_cache
import random
import requests
from requests import Session

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statbotics

%matplotlib notebook

sb = statbotics.Statbotics()

In [4]:
all_teams = sb.get_team_years(year=2023, limit=10000)
all_teams_dict = {t["team"]: t for t in all_teams}
len(all_teams)

3286

In [17]:
AUTH_KEY = "XeUIxlvO4CPc44NlLE3ncevDg7bAhp6CRy6zC9M2aQb2zGfys0M30eKwavFJSEJr"

read_prefix = "https://www.thebluealliance.com/api/v3/"

session = Session()
session.headers.update({"X-TBA-Auth-Key": AUTH_KEY, "X-TBA-Auth-Id": ""})

def get_tba(url):
    response = session.get(read_prefix + url)
    return response.json()

@lru_cache()
def get_event_teams(key):
    data = get_tba(f"event/{key}/teams")
    return [int(x["key"][3:]) for x in data]

In [38]:
divisions = [
    get_event_teams("2023arc"),
    get_event_teams("2023cur"),
    get_event_teams("2023dal"),
    get_event_teams("2023gal"),
    get_event_teams("2023hop"),
    get_event_teams("2023joh"),
    get_event_teams("2023mil"),
    get_event_teams("2023new"),
]

champ_team_nums = []
for division in divisions:
    champ_team_nums.extend(division)

In [11]:
epa_breakdown = pd.read_json("https://raw.githubusercontent.com/avgupta456/statbotics/master/data/2023/epa_breakdown.json", orient="index")


In [12]:
team_to_epa = {t: all_teams_dict[t]["epa_end"] for t in champ_team_nums}
team_to_rp_1_epa = {t: all_teams_dict[t]["rp_1_epa_end"] for t in champ_team_nums}
team_to_rp_2_epa = {t: all_teams_dict[t]["rp_2_epa_end"] for t in champ_team_nums}
team_to_cycles = {t: epa_breakdown.loc[t]["total_cycles"] for t in champ_team_nums}

print(team_to_epa[2056], team_to_rp_1_epa[2056], team_to_rp_2_epa[2056], team_to_cycles[2056])
print(team_to_epa[254], team_to_rp_1_epa[254], team_to_rp_2_epa[254], team_to_cycles[254])

84.85 0.9931 0.5175 12.2601407515
80.19 0.9545 0.6071 11.6093456675


In [13]:
TOTAL_MEAN = 74.57
TOTAL_SD = 29.36

In [14]:
@lru_cache
def make_request(url):
    response = requests.get(url)
    response.raise_for_status()
    data = response.text
    lines = data.split("\n")
    return lines

@lru_cache
def get_schedule(num_teams: int, num_matches: int):
    # TODO: remove this once we have pre-generated schedules for 100+ teams
    if num_teams > 100:
        schedule1 = get_schedule(100, num_matches)
        schedule2 = get_schedule(num_teams - 100, num_matches)
        schedule2 = [
            {"red": [team + 100 for team in match["red"]], "blue": [team + 100 for team in match["blue"]]}
            for match in schedule2
        ]
        return schedule1 + schedule2

    # load csv from external URL using requests
    lines = make_request(f"https://raw.githubusercontent.com/Team254/cheesy-arena/main/schedules/{num_teams}_{num_matches}.csv")
    
    schedule = []
    for line in lines:
        match = line.split(",")
        if len(match) < 12:
            continue
        red = [int(match[0]), int(match[2]), int(match[4])]
        blue = [int(match[6]), int(match[8]), int(match[10])]
        schedule.append({"red": red, "blue": blue})
        
    return schedule

In [56]:
def get_win_prob(a, b):
    return 1 / (1 + 10 ** (((-5 / 8) * (a - b)) / TOTAL_SD))

def get_rp_pred(a):
    return 1 / (1 + np.e ** (-4 * (a - 0.5)))

def sim_single_quals(teams, schedule, team_to_epa, team_to_rp_1_epa, team_to_rp_2_epa):
    curr_sim_matches = {t: 0 for t in teams}
    curr_sim_rps = {t: 0 for t in teams}
    for m in schedule:
        red_epa = sum(team_to_epa[x] for x in m["red"])
        blue_epa = sum(team_to_epa[x] for x in m["blue"])
        red_rp_1_epa = sum(team_to_rp_1_epa[x] for x in m["red"])
        blue_rp_1_epa = sum(team_to_rp_1_epa[x] for x in m["blue"])
        red_rp_2_epa = sum(team_to_rp_2_epa[x] for x in m["red"])
        blue_rp_2_epa = sum(team_to_rp_2_epa[x] for x in m["blue"])
        
        win_prob = get_win_prob(red_epa, blue_epa)
        red_win = 1 if random.random() < win_prob else 0

        red_rp_1_prob = get_rp_pred(0.9 * red_rp_1_epa)
        red_rp_1 = 1 if random.random() < red_rp_1_prob else 0
        
        red_rp_2_prob = get_rp_pred(red_rp_2_epa)
        red_rp_2 = 1 if random.random() < red_rp_2_prob else 0
        
        blue_rp_1_prob = get_rp_pred(0.9 * blue_rp_1_epa)
        blue_rp_1 = 1 if random.random() < blue_rp_1_prob else 0
        
        blue_rp_2_prob = get_rp_pred(blue_rp_2_epa)
        blue_rp_2 = 1 if random.random() < blue_rp_2_prob else 0

        red_rps = red_rp_1 + red_rp_2 + (2 if red_win else 0)
        blue_rps = blue_rp_1 + blue_rp_2 + (0 if red_win else 2)
        
        for x in m["red"]:
            curr_sim_matches[x] += 1
            if curr_sim_matches[x] <= 10:
                curr_sim_rps[x] += red_rps
            
        for x in m["blue"]:
            curr_sim_matches[x] += 1
            if curr_sim_matches[x] <= 10:
                curr_sim_rps[x] += blue_rps
            
    curr_sim_ranks = sorted(curr_sim_rps.items(), key=lambda x: [-x[1], random.random()])
    
    return [x[0] for x in curr_sim_ranks]

def softmax_select(options, mult=1):
    exp = [np.e ** (mult * o) for o in options]
    sum_exp = sum(exp)
    exp = [e / sum_exp for e in exp]
    rand = random.random()
    curr, i = 0, 0
    while curr < rand:
        curr += exp[i]
        i += 1
    return i - 1

def sim_alliance_selection(ranks, team_to_epa):
    alliances = [[], [], [], [], [], [], [], []]
    locked_teams = []
    remaining_teams = [(r, team_to_epa[r]) for r in ranks]
    
    r1_mult = 1 / 3
    r2_mult = 1 / 2
    
    def handle_selection(selector, reject=True):
        selected = None
        while selected is None:
            temp_remaining_teams = [r for r in remaining_teams if r not in locked_teams]
            _selected = softmax_select([r[1] for r in temp_remaining_teams], r1_mult)
            _selected_team = temp_remaining_teams[_selected]
            orig_rank = ranks.index(_selected_team[0]) + 1
            if not reject or orig_rank > 8:
                selected = _selected
                continue
                
            selected_rank = remaining_teams.index(_selected_team) + 1
            remaining_best_epas = sorted(temp_remaining_teams[_selected + 1:], key=lambda x: -x[1])
            if remaining_best_epas[selected_rank][1] > selector[1] + 5:
                locked_teams.append(_selected_team)
            else:
                selected = _selected
                
        return remaining_teams.index(temp_remaining_teams[selected])
    
    # Captain and Round 1
    alliances[0].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[0].append(remaining_teams.pop(selected))
    
    alliances[1].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[1].append(remaining_teams.pop(selected))
    
    alliances[2].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[2].append(remaining_teams.pop(selected))
    
    alliances[3].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[3].append(remaining_teams.pop(selected))

    alliances[4].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[4].append(remaining_teams.pop(selected))
    
    alliances[5].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[5].append(remaining_teams.pop(selected))
    
    alliances[6].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[6].append(remaining_teams.pop(selected))
    
    alliances[7].append(remaining_teams.pop(0))
    selected = handle_selection(alliances[0][0])
    alliances[7].append(remaining_teams.pop(selected))
    
    # Round 2
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[7].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[6].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[5].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[4].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[3].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[2].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[1].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[0].append(remaining_teams.pop(selected))
    
    # Round 3
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[0].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[1].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[2].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[3].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[4].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[5].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[6].append(remaining_teams.pop(selected))
    selected = softmax_select([r[1] for r in remaining_teams], r2_mult)
    alliances[7].append(remaining_teams.pop(selected))
    
    aepas = [sum(a[1] for a in alliance[:3]) for alliance in alliances]
    return [[a[0] for a in alliance] for alliance in alliances], aepas

def sim_single_elims(alliances, team_to_epa):
    scores, grids = [], []
    
    aepas = [sum(team_to_epa[a] for a in alliance[:3]) for alliance in alliances]
    ms = [[0, 7], [3, 4], [1, 6], [2, 5], [], [], [], [], [], [], [], [], []]
    
    def _get_win_prob(i):
        return get_win_prob(aepas[ms[i][0]], aepas[ms[i][1]])

    m1_red_winner = random.random() < _get_win_prob(0)
    ms[4].append(ms[0][1] if m1_red_winner else ms[0][0])
    ms[6].append(ms[0][0] if m1_red_winner else ms[0][1])
    
    m2_red_winner = random.random() < _get_win_prob(1)
    ms[4].append(ms[1][1] if m2_red_winner else ms[1][0])
    ms[6].append(ms[1][0] if m2_red_winner else ms[1][1])
    
    m3_red_winner = random.random() < _get_win_prob(2)
    ms[5].append(ms[2][1] if m3_red_winner else ms[2][0])
    ms[7].append(ms[2][0] if m3_red_winner else ms[2][1])
    
    m4_red_winner = random.random() < _get_win_prob(3)
    ms[5].append(ms[3][1] if m4_red_winner else ms[3][0])
    ms[7].append(ms[3][0] if m4_red_winner else ms[3][1])
    
    m5_red_winner = random.random() < _get_win_prob(4)
    ms[9].append(ms[4][0] if m5_red_winner else ms[4][1])
    
    m6_red_winner = random.random() < _get_win_prob(5)
    ms[8].append(ms[5][0] if m6_red_winner else ms[5][1])
    
    m7_red_winner = random.random() < _get_win_prob(6)
    ms[8].append(ms[6][1] if m7_red_winner else ms[6][0])
    ms[10].append(ms[6][0] if m7_red_winner else ms[6][1])
    
    m8_red_winner = random.random() < _get_win_prob(7)
    ms[9].append(ms[7][1] if m8_red_winner else ms[7][0])
    ms[10].append(ms[7][0] if m8_red_winner else ms[7][1])
    
    m9_red_winner = random.random() < _get_win_prob(8)
    ms[11].append(ms[8][0] if m9_red_winner else ms[8][1])
    
    m10_red_winner = random.random() < _get_win_prob(9)
    ms[11].append(ms[9][0] if m10_red_winner else ms[9][1])
    
    m11_red_winner = random.random() < _get_win_prob(10)
    ms[12].append(ms[10][1] if m11_red_winner else ms[10][0])
    finalist_1 = ms[10][0] if m11_red_winner else ms[10][1]
    
    m12_red_winner = random.random() < _get_win_prob(11)
    fourth_place = ms[11][1] if m12_red_winner else ms[11][0]
    ms[12].append(ms[11][0] if m12_red_winner else ms[11][1])
    
    m13_red_winner = random.random() < _get_win_prob(12)
    third_place = ms[12][1] if m13_red_winner else ms[12][0]
    finalist_2 = ms[12][0] if m13_red_winner else ms[12][1]
    
    
    f1_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    f2_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    f3_red_winner = random.random() < get_win_prob(aepas[finalist_1], aepas[finalist_2])
    winner = finalist_1 if f1_red_winner + f2_red_winner + f3_red_winner >= 2 else finalist_2
    second_place = finalist_2 if winner == finalist_1 else finalist_1
    
    return winner, second_place, third_place, fourth_place


In [63]:
overall_winner_count = defaultdict(int)
overall_finalist_count = defaultdict(int)
overall_third_place_count = defaultdict(int)
overall_fourth_place_count = defaultdict(int)

winner_count = defaultdict(int)
finalist_count = defaultdict(int)
third_place_count = defaultdict(int)
fourth_place_count = defaultdict(int)

winner_alliances = defaultdict(int)
winner_best_epas = []
alliance_epas = []
best_epas = []
winner_epas = []

einstein_winner_best_epas = []
einstein_alliance_epas = []
einstein_best_epas = []
einstein_winner_epas = []

einstein_winner = defaultdict(int)
einstein_finalist = defaultdict(int)
einstein_third = defaultdict(int)
einstein_fourth = defaultdict(int)

einstein_pairs = defaultdict(int)
winning_pairs = defaultdict(int)

qual_rank_list = defaultdict(lambda: [0] * 78)

num_sims = 10000

def pre_sim():
    for sim in range(num_sims):
        if sim % 100 == 0:
            print(sim)
            
        noise = 4.5
        curr_team_to_epa = {k : v + np.random.normal(0, noise) for k, v in team_to_epa.items()}
        
        rp_noise = 0.1
        curr_team_to_rp_1_epa = {k : v + np.random.normal(0, rp_noise) for k, v in team_to_rp_1_epa.items()}
        curr_team_to_rp_2_epa = {k : v + np.random.normal(0, rp_noise) for k, v in team_to_rp_2_epa.items()}
        
        einstein_alliances = []
        einstein_aepas = []
        for division in divisions:
            random.shuffle(division)
            schedule = get_schedule(len(division), 10)
            schedule = [
                {
                    "red": [division[x - 1] for x in m["red"]],
                    "blue": [division[x - 1] for x in m["blue"]],
                }
                for m in schedule
            ]
            qual_ranks = sim_single_quals(division, schedule, curr_team_to_epa, curr_team_to_rp_1_epa, curr_team_to_rp_2_epa)
            for i, t in enumerate(qual_ranks):
                qual_rank_list[t][i] += 1
            
            alliances, aepas = sim_alliance_selection(qual_ranks, curr_team_to_epa)
            winner, finalist, third_place, fourth_place = sim_single_elims(alliances, curr_team_to_epa)
            einstein_alliances.append(alliances[winner])
            einstein_aepas.append(aepas[winner])
            
            einstein_pairs[frozenset((alliances[winner][0], alliances[winner][1]))] += 1
            
            winner_alliances[winner] += 1
            best_epa = max(aepas)
            winner_epa = aepas[winner]
            winner_best_epas.append(best_epa == winner_epa)
            alliance_epas.extend(aepas)
            best_epas.append(best_epa)
            winner_epas.append(winner_epa)
            
            for t in alliances[winner]:
                winner_count[t] += 1
                
            for t in alliances[finalist]:
                finalist_count[t] += 1
                
            for t in alliances[third_place]:
                third_place_count[t] += 1
                
            for t in alliances[fourth_place]:
                fourth_place_count[t] += 1
            
        winner, finalist, third_place, fourth_place = sim_single_elims(einstein_alliances, curr_team_to_epa)
        einstein_winner[winner] += 1
        einstein_finalist[finalist] += 1
        einstein_third[third_place] += 1
        einstein_fourth[fourth_place] += 1
        
        winning_pairs[frozenset((einstein_alliances[winner][0], einstein_alliances[winner][1]))] += 1
        
        einstein_best_epa = max(einstein_aepas)
        einstein_winner_epa = einstein_aepas[winner]
        einstein_winner_best_epas.append(einstein_best_epa == einstein_winner_epa)
        einstein_alliance_epas.extend(einstein_aepas)
        einstein_best_epas.append(einstein_best_epa)
        einstein_winner_epas.append(einstein_winner_epa)

        for t in einstein_alliances[winner]:
            overall_winner_count[t] += 1
            
        for t in einstein_alliances[finalist]:
            overall_finalist_count[t] += 1
            
        for t in einstein_alliances[third_place]:
            overall_third_place_count[t] += 1
            
        for t in einstein_alliances[fourth_place]:
            overall_fourth_place_count[t] += 1
            
pre_sim()


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [64]:
print(sorted(einstein_pairs.items(), key=lambda x: -x[1])[:10])
print(sorted(winning_pairs.items(), key=lambda x: -x[1])[:10])

[(frozenset({2056, 254}), 2772), (frozenset({3005, 1678}), 1345), (frozenset({1323, 1987}), 1115), (frozenset({3538, 195}), 831), (frozenset({2338, 1323}), 820), (frozenset({1323, 4414}), 813), (frozenset({971, 7157}), 750), (frozenset({2056, 111}), 664), (frozenset({2056, 6036}), 593), (frozenset({1706, 1323}), 520)]
[(frozenset({2056, 254}), 2101), (frozenset({2056, 111}), 420), (frozenset({3005, 1678}), 417), (frozenset({2056, 6036}), 352), (frozenset({1323, 1987}), 264), (frozenset({2338, 1323}), 200), (frozenset({254, 111}), 194), (frozenset({2056, 1619}), 181), (frozenset({6036, 254}), 181), (frozenset({1323, 4414}), 155)]


In [68]:
def pad(s, length = 15):
    return s + "&nbsp;" * (length - len(s))

division_names = ["Archimedes", "Curie", "Daly", "Galileo", "Hopper", "Johnson", "Milstein", "Newton"]

print("| Alliance | Win Probability |")
print("| --- | --- |")
for i, count in sorted(winner_alliances.items(), key=lambda x: -x[1]):
    print("|", i + 1, "|", str(round(100 * count / 8 / num_sims, 2)) + "%", "|")
print()

print("| Division | ", pad("Winner"), " | ", pad("Finalist"), " | ", pad("Third"), " | ", pad("Fourth"), " |")
print("| --- | --- | --- | --- | --- |")
for i, count in sorted(einstein_winner.items(), key=lambda x: x[0]):
    print("|", division_names[i], "|", str(round(100 * count / num_sims, 2)) + "%", "|", str(round(100 * einstein_finalist[i] / num_sims, 2)) + "%", "|", str(round(100 * einstein_third[i] / num_sims, 2)) + "%", "|", str(round(100 * einstein_fourth[i] / num_sims, 2)) + "%", "|")
print()


| Alliance | Win Probability |
| --- | --- |
| 1 | 33.98% |
| 2 | 17.83% |
| 3 | 12.45% |
| 4 | 9.62% |
| 5 | 8.07% |
| 6 | 6.71% |
| 7 | 6.02% |
| 8 | 5.33% |

| Division |  Winner&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Finalist&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Third&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Fourth&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |
| --- | --- | --- | --- | --- |
| Archimedes | 50.05% | 13.37% | 9.84% | 8.44% |
| Curie | 4.09% | 12.14% | 12.99% | 11.22% |
| Daly | 2.36% | 9.92% | 13.14% | 10.71% |
| Galileo | 12.68% | 13.57% | 12.28% | 16.3% |
| Hopper | 12.9% | 12.29% | 12.23% | 16.26% |
| Johnson | 5.95% | 14.28% | 15.32% | 11.49% |
| Milstein | 8.38% | 17.99% | 16.75% | 11.79% |
| Newton | 3.59% | 6.44% | 7.45% | 13.79% |



In [69]:
avg_ranks = {t: sum(((i + 1) * qual_rank_list[t][i]) for i in range(78)) / num_sims for t in qual_rank_list}
top_1 = {t: sum(qual_rank_list[t][i] for i in range(1)) / num_sims for t in qual_rank_list}
top_4 = {t: sum(qual_rank_list[t][i] for i in range(4)) / num_sims for t in qual_rank_list}
top_8 = {t: sum(qual_rank_list[t][i] for i in range(8)) / num_sims for t in qual_rank_list}
top_16 = {t: sum(qual_rank_list[t][i] for i in range(16)) / num_sims for t in qual_rank_list}

def print_stats(filter=lambda x: True, limit=25):
    print("| Rank | ", pad("Team"), " | ", pad("Mean Rank"), " | ", pad("1st"), " | ", pad("Top 4"), " | ", pad("Top 8"), " | ", pad("Top 16"), " |")
    print("| --- | --- | --- | --- | --- | --- | --- |")
    for i, (key, avg_rank) in enumerate(sorted(avg_ranks.items(), key=lambda x: (filter(x[0]), x[1]))[:limit]):
        print("|", i + 1, "|", key, "|", round(avg_rank, 2), "|", str(round(100 * top_1[key], 1)) + "%", "|", str(round(100 * top_4[key], 1)) + "%", "|", str(round(100 * top_8[key], 1)) + "%", "|", str(round(100 * top_16[key], 1)) + "%", "|")
    print()

    print("Division")
    print("| Rank | ", pad("Team"), " | ", pad("Winner"), " | ", pad("Finalist"), " | ", pad("3rd Place"), " | ", pad("4th Place"), " |")
    print("| ---- | ---- | --- | --- | --- | --- |")
    for i, (key, count) in enumerate(sorted(winner_count.items(), key=lambda x: (filter(x[0]), -x[1]))[:limit]):
        print("|", i + 1, "|", key, "|", str(round(100 * winner_count[key] / num_sims, 1)) + "%", "|", str(round(100 * finalist_count[key] / num_sims, 1)) + "%", "|", str(round(100 * third_place_count[key] / num_sims, 1)) + "%", "|", str(round(100 * fourth_place_count[key] / num_sims, 1)) + "%", "|")
    print()

    print("Einstein")
    print("| Rank | ", pad("Team"), " | ", pad("Winner"), " | ", pad("Finalist"), " | ", pad("3rd Place"), " | ", pad("4th Place"), " |")
    print("| ---- | ---- | --- | --- | --- | --- |")
    for i, (key, count) in enumerate(sorted(overall_winner_count.items(), key=lambda x: (filter(x[0]), -x[1]))[:limit]):
        print("|", i + 1, "|", key, "|", str(round(100 * overall_winner_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_finalist_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_third_place_count[key] / num_sims, 1)) + "%", "|", str(round(100 * overall_fourth_place_count[key] / num_sims, 1)) + "%", "|")
    print()
    
print_stats()


| Rank |  Team&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Mean Rank&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  1st&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 8&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 16&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | 1323 | 5.24 | 28.3% | 62.0% | 80.8% | 94.7% |
| 2 | 2056 | 6.75 | 21.0% | 53.1% | 73.5% | 90.0% |
| 3 | 254 | 7.8 | 17.2% | 47.2% | 68.3% | 87.0% |
| 4 | 3538 | 7.99 | 18.7% | 47.7% | 68.3% | 85.9% |
| 5 | 1678 | 8.11 | 18.9% | 46.9% | 67.1% | 85.8% |
| 6 | 195 | 9.43 | 14.8% | 40.9% | 61.1% | 81.4% |
| 7 | 971 | 10.03 | 14.0% | 38.9% | 58.8% | 80.3% |
| 8 | 1987 | 10.44 | 9.3% | 32.5% | 54.4% | 79.0% |
| 9 | 5940 | 10.75 | 12.9% | 36.8% | 55.4% | 77.2% |
| 10 | 4414 | 11.09 | 8.0% | 30.8% | 51.9% | 76.9% |
| 11 | 3005 | 11.7

In [70]:
for i in range(8):
    print(division_names[i])
    print_stats(filter=lambda x: 0 if x in divisions[i] else 1, limit=16)
    print()
    print()

Archimedes
| Rank |  Team&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Mean Rank&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  1st&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 8&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |  Top 16&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  |
| --- | --- | --- | --- | --- | --- | --- |
| 1 | 2056 | 6.75 | 21.0% | 53.1% | 73.5% | 90.0% |
| 2 | 254 | 7.8 | 17.2% | 47.2% | 68.3% | 87.0% |
| 3 | 6036 | 13.55 | 6.7% | 24.8% | 43.9% | 68.6% |
| 4 | 111 | 13.79 | 6.8% | 24.9% | 43.0% | 67.8% |
| 5 | 1619 | 16.17 | 4.6% | 18.7% | 35.1% | 59.8% |
| 6 | 1577 | 16.55 | 4.2% | 17.4% | 34.0% | 58.8% |
| 7 | 2468 | 16.61 | 3.9% | 17.5% | 33.6% | 58.5% |
| 8 | 118 | 17.32 | 3.5% | 14.9% | 30.2% | 55.9% |
| 9 | 2687 | 18.26 | 3.2% | 14.5% | 29.0% | 53.9% |
| 10 | 4028 | 18.38 | 3.3% | 14.7% | 28.7% | 53.4% |
| 11 | 30

In [71]:
def get_stats(team):
    print("Ranks", round(avg_ranks[team], 2), str(round(100 * top_1[team], 1)) + "%", str(round(100 * top_4[team], 1)) + "%", str(round(100 * top_8[team], 1)) + "%", str(round(100 * top_16[team], 1)) + "%")
    print("Division", str(round(100 * winner_count[team] / num_sims, 1)) + "%", str(round(100 * finalist_count[team] / num_sims, 1)) + "%", str(round(100 * third_place_count[team] / num_sims, 1)) + "%", str(round(100 * fourth_place_count[team] / num_sims, 1)) + "%")
    print("Einstein", str(round(100 * overall_winner_count[team] / num_sims, 1)) + "%", str(round(100 * overall_finalist_count[team] / num_sims, 1)) + "%", str(round(100 * overall_third_place_count[team] / num_sims, 1)) + "%", str(round(100 * overall_fourth_place_count[team] / num_sims, 1)) + "%")  
          
get_stats(254)
          

Ranks 7.8 17.2% 47.2% 68.3% 87.0%
Division 47.3% 17.9% 11.4% 7.1%
Einstein 29.3% 5.8% 4.0% 3.1%
